In [ ]:
import jax.numpy as jnp

from qpm import cwes, mgoslt

widths = jnp.load("optimized_widths_1000.npy")
kappa_vals = jnp.ones(1000) * 1.31e-4 / (2 / jnp.pi)
delta_k1 = mgoslt.calc_twm_delta_k(wls, wls, 70.0)
delta_k2 = mgoslt.calc_twm_delta_k(wls, wls / 2, 70.0)
b_initial = jnp.array([1.0, 0.0, 0.0])
cwes.simulate_twm(widths, kappa_vals, delta_k1, delta_k2, b_initial)
